In [1]:
import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc

ColumnNames = ["startDate", "endDate", "value"]

globalPath = "/home/chowder/Documents/personalData/ouraRing/"
jsonFileName = "oura_2021-01-16T02-12-49.json"

parquetName = "ouraRingHypnoDf.parquet.gzip"

with open(globalPath + jsonFileName) as f:
  jsonData = json.load(f)

1.1.2


In [2]:
def ouraSleepStageToStandard(ouraNum):
    if ouraNum == 4: #awake
        return 0
    if ouraNum == 2: #light sleep
        return 1
    if ouraNum == 1: #deep sleep
        return 2
    if ouraNum == 3: #REM sleep
        return 3
    

In [3]:
print(jsonData['sleep'][0]['bedtime_start'])
print(jsonData['sleep'][0]['bedtime_end'])
print(jsonData['sleep'][0]['hypnogram_5min'])
print(len(jsonData['sleep'][0]['hypnogram_5min']))
print(pd.to_datetime(jsonData['sleep'][0]['bedtime_start']) + datetime.timedelta(minutes=len(jsonData['sleep'][0]['hypnogram_5min'])*5))


2020-10-15T01:03:59-07:00
2020-10-15T09:47:59-07:00
424221114322222121112211123322211111122222211223312224333333242221122222333333442222222223333344444444444
105
2020-10-15 09:48:59-07:00


In [4]:
listRecords = []
startTime = pd.to_datetime(jsonData['sleep'][0]['bedtime_start'])

hypnogram_5min = jsonData['sleep'][0]['hypnogram_5min']
toadd = []
toadd.append(startTime)

for i in range(1, len(hypnogram_5min)):
    if(hypnogram_5min[i-1] != hypnogram_5min[i]):#if the block has ended
        toadd.append(startTime + datetime.timedelta(minutes=i*5)) # add the end time
        toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[i-1])))
        listRecords.append(toadd)
        toadd = [startTime + datetime.timedelta(minutes=i*5)]

toadd.append(startTime + datetime.timedelta(minutes=len(hypnogram_5min)*5))
toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[-1])))
listRecords.append(toadd)

In [5]:
listRecords[-1]

[Timestamp('2020-10-15 08:53:59-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2020-10-15 09:48:59-0700', tz='pytz.FixedOffset(-420)'),
 0]

In [6]:
listRecords = []

for SleepJSON in jsonData['sleep']:
    startTime = pd.to_datetime(SleepJSON['bedtime_start'])
    stageStartTime = startTime

    hypnogram_5min = SleepJSON['hypnogram_5min']
    toadd = []
    toadd.append(stageStartTime)

    for i in range(1, len(hypnogram_5min)):
        if(hypnogram_5min[i-1] != hypnogram_5min[i]):#if the block has ended
            toadd.append(startTime + datetime.timedelta(minutes=i*5)) # add the end time
            toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[i-1])))
            listRecords.append(toadd)
            toadd = [startTime + datetime.timedelta(minutes=i*5)]
    
    toadd.append(startTime + datetime.timedelta(minutes=len(hypnogram_5min)*5))
    toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[-1])))
    listRecords.append(toadd)




In [7]:
len(listRecords)

2424

In [8]:
for restfulJSON in jsonData['restful_periods']:
    startTime = pd.to_datetime(restfulJSON['bedtime_start'])
    endTime = pd.to_datetime(restfulJSON['bedtime_end'])
    listRecords.append([startTime, endTime, 1]) #fill in naps with light sleep

In [9]:
len(listRecords)

2511

In [10]:
listRecords = sorted(listRecords, key=lambda x:x[0])


In [11]:
# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(listRecords)-1):
    filledRecords.append(listRecords[ri])
    # if there is a gap between this rows end date and the next rows start date
    if listRecords[ri][1] < listRecords[ri+1][0]:
        filledRecords.append([
            listRecords[ri][1],
            listRecords[ri+1][0],
            -1
        ])
print(len(filledRecords))

2684


In [12]:
set([type(x[1]) for x in filledRecords])

{pandas._libs.tslibs.timestamps.Timestamp}

In [13]:
ouraHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)

ouraHypnoDf = ouraHypnoDf.set_index("startDate")
ouraHypnoDf

,endDate,value
startDate,,
2020-10-15 01:03:59-07:00,2020-10-15 01:08:59-07:00,0
2020-10-15 01:08:59-07:00,2020-10-15 01:13:59-07:00,1
2020-10-15 01:13:59-07:00,2020-10-15 01:18:59-07:00,0
2020-10-15 01:18:59-07:00,2020-10-15 01:28:59-07:00,1
2020-10-15 01:28:59-07:00,2020-10-15 01:43:59-07:00,2
...,...,...
2021-01-15 06:56:19-07:00,2021-01-15 07:01:19-07:00,3
2021-01-15 07:01:19-07:00,2021-01-15 07:41:19-07:00,1
2021-01-15 07:41:19-07:00,2021-01-15 08:26:19-07:00,3


In [14]:
ouraHypnoDf.dtypes

endDate    object
value       int64
dtype: object

In [15]:
ouraHypnoDf["endDate"]

startDate
2020-10-15 01:03:59-07:00    2020-10-15 01:08:59-07:00
2020-10-15 01:08:59-07:00    2020-10-15 01:13:59-07:00
2020-10-15 01:13:59-07:00    2020-10-15 01:18:59-07:00
2020-10-15 01:18:59-07:00    2020-10-15 01:28:59-07:00
2020-10-15 01:28:59-07:00    2020-10-15 01:43:59-07:00
                                       ...            
2021-01-15 06:56:19-07:00    2021-01-15 07:01:19-07:00
2021-01-15 07:01:19-07:00    2021-01-15 07:41:19-07:00
2021-01-15 07:41:19-07:00    2021-01-15 08:26:19-07:00
2021-01-15 08:26:19-07:00    2021-01-15 08:31:19-07:00
2021-01-15 08:31:19-07:00    2021-01-15 08:51:19-07:00
Name: endDate, Length: 2684, dtype: object

In [16]:
ouraHypnoDf.to_parquet(globalPath + parquetName,
              compression='gzip') 

NameError: name 'parquetName' is not defined